### --- Day 19: Aplenty ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/19

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 19);
WriteLines(inputLines);

Loading puzzle file: Day19.txt
Total lines: 771
Max line length: 40

vm{a>1183:zz,s>1553:A,s<827:xc,hls}
rvv{m<3430:A,m>3663:A,a<1488:mnf,prg}
px{s>1361:cfv,x>2479:R,R}
pz{x<2570:R,s>1531:ccq,x<2716:dgp,R}
sh{m>1073:A,s>1756:R,m<1022:A,A}


In [4]:
class Workflow(string id, IEnumerable<Stage> stages) 
{
    public string Id { get; } = id;

    public List<Stage> Stages { get; } = stages.ToList(); // public for part 2

    public string GetWorkflow(Part part) => Stages.Where(st => st.Matches(part)).First().Workflow;

    public override string ToString() => $"{Id}{{{string.Join(',', Stages)}}}";
}

abstract class Stage(string workflow) {
    public string Workflow { get; } = workflow;

    public abstract bool Matches(Part part);
}

class DefaultStage(string workflow) : Stage(workflow) {
    public override bool Matches(Part part) => true;

    public override string ToString() => Workflow;
}

class ComparisonStage(string workflow, char attribute, char comparison, int value) : Stage(workflow) {
    public override bool Matches(Part part) => Comparison switch {
        '>' => part[Attribute] > Value,
        '<' => part[Attribute] < Value,
        _ => throw new Exception("Unrecognised comparison")
    };

    public override string ToString() => $"{Attribute}{Comparison}{Value}:{Workflow}";

    // The following required for part 2

    public char Attribute {get;} = attribute;
    public char Comparison {get;} = comparison;
    public int Value { get; } = value;
}

class Part : Dictionary<char, int> {
    public Part(int x, int m, int a, int s) {
        this['x'] = x;
        this['m'] = m;
        this['a'] = a;
        this['s'] = s;
    }

    public override string ToString() => $"{{x={this['x']},m={this['m']},a={this['a']},s={this['s']}}}";

    public int Rating => "xmas".Select(ch => this[ch]).Sum();
}

class Sorter(IEnumerable<Workflow> workflows) {
    public Dictionary<string, Workflow> Workflows { get; } = workflows.ToDictionary(wf => wf.Id);

    bool IsAccepted(Part part) {
        var nextWorkflow = "in";
        var safety = 0;

        while (safety < 100) {
            if (nextWorkflow == "A") return true;
            if (nextWorkflow == "R") return false;

            nextWorkflow = Workflows[nextWorkflow].GetWorkflow(part);
            safety++;
        }

        throw new Exception("Safety limit reached");
    }

    public int Sort(IEnumerable<Part> parts) => parts.Where(IsAccepted).Select(p => p.Rating).Sum();
}

In [5]:
using System.Text.RegularExpressions;

Stage ParseStage(string s) {
    var comp = new Regex(@"([xmas])([<>])(\d+):(\w+)");

    var compMatch = comp.Match(s);
    if (compMatch.Success) {
        var attribute = compMatch.Groups[1].Value[0];
        var comparison = compMatch.Groups[2].Value[0];
        var value = int.Parse(compMatch.Groups[3].Value);
        var workflow = compMatch.Groups[4].Value;

        return new ComparisonStage(workflow, attribute, comparison, value);
    } else {
        return new DefaultStage(s);
    }
}

Workflow ParseWorkflow(string line) {
    var lineBits = line.Split('{');
    var id = lineBits[0];
    var stages = lineBits[1].Substring(0, lineBits[1].Length - 1).Split(',').Select(ParseStage);

    return new Workflow(id, stages);
}

In [6]:
var testInputWorkflowStr = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}
""";
var testInputWorkflowLines = testInputWorkflowStr.Split('\n').ToList();

var testInputWorkflows = testInputWorkflowLines.Select(ParseWorkflow).ToList();
foreach (var wf in testInputWorkflows.Take(5)) {
    Console.WriteLine(wf);
}

px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}


In [7]:
var testInputSorter = new Sorter(testInputWorkflows);

In [8]:
var partRegex = new Regex(@"\w=(\d+)");

Part ParsePart(string line) {
    var a = partRegex.Matches(line).Select(m => int.Parse(m.Groups[1].Value)).ToArray();

    return new Part(a[0], a[1], a[2], a[3]);
}

In [9]:
var testInputPartStr = """
{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
""";
var testInputPartLines = testInputPartStr.Split('\n').ToArray();
var testInputParts = testInputPartLines.Select(ParsePart).ToList();
foreach (var p in testInputParts) {
    Console.WriteLine(p);
}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}


In [10]:
// Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

var testInputAnswer = testInputSorter.Sort(testInputParts);
Console.WriteLine(testInputAnswer);

19114


In [11]:
// Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?

var inputWorkflows = inputLines.TakeWhile(line => line != "").Select(ParseWorkflow);
var inputParts = inputLines.SkipWhile(line => line != "").Skip(1).Select(ParsePart);
var inputSorter = new Sorter(inputWorkflows);

var part1Answer = inputSorter.Sort(inputParts);
Console.WriteLine(part1Answer);

401674


In [12]:
// 401674 is correct!
Ensure(401674, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/19

In [14]:
record MinMax(int min = 1, int max = 4000) {
    public int Combos => max - min + 1;
}

class Avail : Dictionary<char, MinMax> {
    public Avail() {
        foreach (var ch in "xmas") { this[ch] = new(); }
    }

    public Avail(Avail clone) : base(clone) {}

    public override string ToString() => $"{{x={this['x']},m={this['m']},a={this['a']},s={this['s']}}}";

    public long Combos => "xmas".Select(ch => (long)this[ch].Combos).Aggregate((a, b) => a * b);
}


In [15]:
IEnumerable<Avail> GetAvails(Sorter s, string workflowId, Avail current) {
    if (workflowId == "A") {
        yield return current;
        yield break;
    }
    else if (workflowId == "R") {
        yield break;
    }
    
    var workflow = s.Workflows[workflowId];

    foreach (var stage in workflow.Stages) {
        if (stage is DefaultStage) {
            foreach (var a2 in GetAvails(s, stage.Workflow, current)) {
                yield return a2;
            }
        } else if (stage is ComparisonStage comp) {
            Avail nextAvail = new(current);
            var nextAvailAttr = nextAvail[comp.Attribute];
            Avail nextCurrent = new(current);
            var nextCurrentAttr = nextCurrent[comp.Attribute];

            switch (comp.Comparison) {
                case '<':
                    nextAvailAttr = nextAvailAttr with { max = comp.Value - 1};
                    nextCurrentAttr = nextCurrentAttr with { min = comp.Value };
                    break;
                case '>':
                    nextAvailAttr = nextAvailAttr with { min = comp.Value + 1};
                    nextCurrentAttr = nextCurrentAttr with { max = comp.Value };
                    break;
                default:
                    throw new Exception("Unexpected comparison");
            }
            nextAvail[comp.Attribute] = nextAvailAttr;
            foreach (var a2 in GetAvails(s, stage.Workflow, nextAvail)) {
                yield return a2;
            }
            nextCurrent[comp.Attribute] = nextCurrentAttr;
            current = nextCurrent;
        }
    }
}

// In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.
var testInputCombos = GetAvails(testInputSorter, "in", new()).Select(a => a.Combos).Sum();
Console.WriteLine(testInputCombos);

167409079868000


In [16]:
// Consider only your list of workflows; the list of part ratings that the Elves
// wanted you to sort is no longer relevant. How many distinct combinations of
// ratings will be accepted by the Elves' workflows?

var part2Answer = GetAvails(inputSorter, "in", new()).Select(a => a.Combos).Sum();
Console.WriteLine(part2Answer);

134906204068564


In [17]:
// 134906204068564 is correct!
Ensure(134906204068564, part2Answer);